# Predicting Building Deterioration in the City of Detroit

## Introduction

In this tutorial, we will explore how to construct a dataset that can be used by machine learning models from publicly avaiable data to solve real problems.

In the City of Detroit, urban planners are fighting **blight** - the deterioration and decay of buildings and older areas due to neglect, crime, or lack of economic support. Our goal in this tutorial is to build a model that can predict the risks (probabilities) of blights, using public incidents datasets provided by the city's open data initiative.

Based on the data available to us, there are two main obstacles to achieving our goal:

- There isn't any established list of all the buildings in the city.
- The data is highly imbalanced due to the relative minority of blighted buildings vs non-blighted buildings.

This tutorial will demonstrate a methodology to solve these obstacles, as well as introduce some tricks involved in the data processing and model training.

The datasets used in this tutorial are:

- `detroit-blight-violations.csv`: records of tickets issued for violations to city rules that involve certain locations.
- `detroit-crimes.csv`: records of incidents of crimes taht involve certain locations.
- `detroit-311.csv`: records of 311 calls (non-emergency incident reporting system) that involve certain locations.
- `detroit-demolition-permits.tsv`: records of permits issued for the demolition of buildings.

All datasets are obtained from Detroit's [Open Data Portal](https://data.detroitmi.gov/). As the files have a size of 188MB in total, I've uploaded them to a Google Drive folder that everyone can access with [this link](https://drive.google.com/open?id=1TeGE8wa8CJ02np8sijDREOT4cOMi-Ryi).

## Parsing and Cleaning Coordinates

In order to train a model that predict how likely a building is to be blighted, we need to have a dataset that contains the features and labels for the buildings. However, there is no explicit definition of "buildings" in the available datasets. Therefore, our first task is to somehow derive a list of all the buildings.

Incident records contain coordinates of locations where the indicents happened. Assuming that each incident corresponds to one or more buildings, we can cluster together all the locations. Each cluster will then be treated as one building. 

As a first step, we will extract coordinates from all the incident records.

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime

In [95]:
data1 = pd.read_csv('data/detroit-311.csv')
data2 = pd.read_csv('data/detroit-blight-violations.csv')
data3 = pd.read_csv('data/detroit-crime.csv')
data4 = pd.read_csv('data/detroit-demolition-permits.tsv', sep='\t')

In [197]:
def parseAddress(dirtyAddress):
    try:
        addressData = dirtyAddress.split('\n')
        lat = addressData[2][1:-1].split(',')[0].strip()
        lng = addressData[2][1:-1].split(',')[1].strip()
        return pd.Series([lat, lng])
    except:
        pass

locationsData =  pd.concat([data['lat'], data['lng']], axis=1)

locationsData2A = data2.apply(lambda row: parseAddress(row['ViolationAddress']), axis=1)
locationsData2B = data2.apply(lambda row: parseAddress(row['MailingAddress']), axis=1)
locationsData2A.columns = ['lat', 'lng']
locationsData2B.columns = ['lat', 'lng']

locationsData3 = pd.concat([data3['LAT'], data3['LON']], axis=1)
locationsData3.columns = ['lat', 'lng']

locationsData4A = data4.apply(lambda row: parseAddress(row['site_location']), axis=1)
locationsData4B = data4.apply(lambda row: parseAddress(row['owner_location']), axis=1)
locationsData4C = data4.apply(lambda row: parseAddress(row['contractor_location']), axis=1)
locationsData4A.columns = ['lat', 'lng']
locationsData4B.columns = ['lat', 'lng']
locationsData4C.columns = ['lat', 'lng']

In [198]:
locations = pd.DataFrame(columns={'lat', 'lng'})

locations = locations.append(locationsData, ignore_index=True)
locations = locations.append(locationsData2A, ignore_index=True)
locations = locations.append(locationsData2B, ignore_index=True)
locations = locations.append(locationsData3, ignore_index=True)
locations = locations.append(locationsData4A, ignore_index=True)
locations = locations.append(locationsData4B, ignore_index=True)
locations = locations.append(locationsData4C, ignore_index=True)

locations['lat'] = pd.to_numeric(locations['lat'], errors='raise') 
locations['lng'] = pd.to_numeric(locations['lng'], errors='raise')

Next, we will clean the coordinates by rounding up their numeric values, dropping duplicates and finally removing outliers. We can use `describe()` method on the dataframe columns to verify that all the coordinates are within the reasonable ranges after the cleaning.

In [200]:
locations['lat'] = locations.apply(lambda row:np.round(row['lat'], 5),axis=1)
locations['lng'] = locations.apply(lambda row:np.round(row['lng'], 5),axis=1)

locations = locations.drop_duplicates()

locations = locations[(locations['lat'] > 23) & (locations['lat'] < 50)]
locations = locations[(locations['lng'] < -64) & (locations['lng'] > -132)]

In [19]:
locations['lat'].describe()

count    264401.000000
mean         42.259709
std           1.149134
min          24.550520
25%          42.359050
50%          42.393900
75%          42.423900
max          48.792170
Name: lat, dtype: float64

In [20]:
locations['lng'].describe()

count    264399.000000
mean        -83.392738
std           3.151343
min        -124.174110
25%         -83.200370
50%         -83.127700
75%         -83.032930
max         -69.298540
Name: lng, dtype: float64

In [24]:
locations = locations.reset_index()

## Clustering Locations to Establish Buildings

Now that we have all the locations, we will use DBSCAN algorithm to cluster together all the locations. Each cluster (bloc) will then be assigned with a unique ID that we will use in the rest part of this tutorial to identify a building.

In [25]:
from sklearn.cluster import DBSCAN
from sklearn.datasets.samples_generator import make_blobs

In [28]:
X = pd.concat([locations['lat'], locations['lng']], axis=1).as_matrix()
db = DBSCAN(eps=0.0001, min_samples=1).fit(X)

core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
print('Number of clusters: %d' % n_clusters_)

Estimated number of clusters: 200969


In [30]:
def populateBlocs(row):
    try:
        return labels[(row.name)]
    except:
        pass

locations['bloc_id'] = locations.apply(populateBlocs, axis=1)
locations = locations.drop('index', axis=1)

Next, we group all the locations by their bloc IDs. Multiple locations could be mapped to the same bloc and thus can be used to derive the points used to define the bloc. A bloc is a rectangular area defined by its central point and 4 corner points. These points can be derived from group statistics givne by `describe()` method.

In [34]:
groupedLocations = locations.groupby('bloc_id', as_index=False)

latitudes = groupedLocations['lat'].describe()
longitudes = groupedLocations['lng'].describe()

latitudes = latitudes.rename(columns={'mean': 'lat_mean', 'min': 'lat_min', 'max': 'lat_max'})
longitudes = longitudes.rename(columns={'mean': 'lng_mean', 'min': 'lng_min', 'max': 'lng_max'})

buildings = pd.concat([latitudes, longitudes], axis=1)

In [38]:
def getBlocBoxes(row):
    return pd.Series([
            row.name, 
            row['lat_mean'], 
            row['lng_mean'], 
            row['lat_min'], 
            row['lat_max'], 
            row['lng_min'], 
            row['lng_max'] 
        ])
    
buildings = buildings.apply(buildings, axis=1)
buildings.columns = ['bloc_id', 'lat_center', 'lng_center', 'lat_min', 'lat_max', 'lng_min', 'lng_max']

buildings.head()

,bloc_id,lat_center,lng_center,lat_min,lat_max,lng_min,lng_max
0,0.0,42.384017,-83.161034,42.38388,42.38415,-83.16107,-83.16095
1,1.0,42.440470,-83.080920,42.44047,42.44047,-83.08092,-83.08092
2,2.0,42.445240,-82.962040,42.44524,42.44524,-82.96204,-82.96204
3,3.0,42.421037,-83.166245,42.42100,42.42106,-83.16630,-83.16619
4,4.0,42.402030,-83.162870,42.40203,42.40203,-83.16287,-83.16287


Here we write a method for getting the list of related bloc IDs given a coordinate. This is done by filtering rows in the `buildings` dataframe we obtained from the last step by checking wether the bloc is close enough to the coordinate given under a certain precision threshold. This method will later be used to associate incident records with bloc IDs.

In [387]:
def getBlocs(x, y):
    go = True
    precision = 1e-08
    
    while (go or len(coords) < 1):
        ret = buildings[(np.isclose(buildings['lat_center'], x, rtol=precision, atol=1e-05)) & (np.isclose(buildings['lng_center'],y, rtol=precision, atol=1e-05))]
        precision *= 10
        if precision >= 1e-04:
            break
        go = False
    
    precision = 1e-08
    while (len(coords) < 1):
        ret = buildings[(np.isclose(buildings['lat_min'], x, rtol=precision, atol=1e-05)) & (np.isclose(buildings['lat_max'],x, rtol=precision, atol=1e-05)) & (np.isclose(buildings['lng_min'], y, rtol=precision, atol=1e-05)) & (np.isclose(buildings['lng_max'],y, rtol=precision, atol=1e-05))]
        precision *= 10
        if precision >= 1e-04:
            return 0

    return list(set(ret['bloc_id']))[0]

Let's test our method.

In [44]:
results = getBlocs(42.440, -83.08)
print(results)

set([41024.0, 187488.0, 45378.0, 43783.0, 75101.0, 150512.0, 135985.0, 191985.0, 60379.0, 35677.0, 40959.0])


## Constructing A Labeled and Balanced Dataset

Now that we have a list of all the buildings and the means to map a coordinate back to its associated building(s), we are going to obtain a subset of the buildings as our final dataset used for the model training.

The number of blighted buildings is relatively small compared to the number of total buildings. Therefore, we need to perform sampling so that the two classes have roughly the same number of samples. A balanced dataset helps with the model training and evaluation.

We treat any building that is associated with one or more demolition permits as a blighted building. Those will be our positive examples (in the context of binary classification). Then we will sample the same number of non-blighted buildings. Those will be our negative examples. Combined they make our labeld and balanced dataset.

First we process the coordinates from the demolition permits dataset.

In [380]:
permits = pd.read_csv('data/detroit-demolition-permits.tsv', sep='\t')

In [382]:
def parseAddress(dirtyAddress):
    try:
        addressData = dirtyAddress.split('\n')
        lat = addressData[2][1:-1].split(',')[0].strip()
        lng = addressData[2][1:-1].split(',')[1].strip()
        return pd.Series([lat, lng])
    except:
        return pd.Series([0, 0])

In [383]:
blightedLocations = permits.apply(lambda row: parseAddress(row['site_location']), axis=1)
blightedLocations.columns = ['lat', 'lng']

blightedLocations = blightedLocations[(blightedLocations['lat'] > 0) & (blightedLocations['lng'] > 0)]

blightedLocations['lat'] = pd.to_numeric(blightedLocations['lat'], errors='raise') 
blightedLocations['lng'] = pd.to_numeric(blightedLocations['lng'], errors='raise') 

Then we obtain the bloc IDs of these samples.

In [388]:
blightedLocations['bloc_id'] = blightedLocations.apply(lambda row: getBlocs(row['lat'], row['lng']), axis=1)

Then we sample from the buildings the same number of blocs as non-blighted samples.

In [498]:
nonBlightedLocations = buildings[~buildings.bloc_id.isin(blightedLocations['bloc_id'])].sample(len(blightedLocations.index))

Finally we combine them into the training dataframe.

In [521]:
trainDF = pd.concat([blightedPlaces['bloc_id']], axis=1)
trainDF['label'] = 'blighted'

nonBlightedLocations = pd.concat([nonBlightedLocations['bloc_id']], axis=1)
nonBlightedLocations['label'] = 'nonblighted'

trainDF = trainDF.append(notBlightedPlaces, ignore_index=True)

## Constructing Features from Incidents Datasets

Now that we have a dataframe that contains the bloc IDs and labels for all the samples, we need to augment it with features that can help predict the blights. In this section, we will parse all the three incident datasets, link each record with buildings it corresponds to and extract all the features we need.

First we will look into the violations dataset.

In [67]:
violations = pd.read_csv('data/detroit-blight-violations.csv')
violations.head()

,TicketID,TicketNumber,AgencyName,ViolName,ViolationStreetNumber,ViolationStreetName,MailingStreetNumber,MailingStreetName,MailingCity,MailingState,MailingZipCode,NonUsAddressCode,Country,TicketIssuedDT,TicketIssuedTime,HearingDT,CourtTime,ViolationCode,ViolDescription,Disposition,FineAmt,AdminFee,LateFee,StateFee,CleanUpCost,JudgmentAmt,PaymentStatus,Void,ViolationCategory,ViolationAddress,MailingAddress
0,26288,05000001DAH,Department of Public Works,"Group, LLC, Grand Holding",2566,GRAND BLVD,743,"Beaubien, Ste. 201",Detroit,MI,48226,NaN,NaN,01/01/38440 12:00:00 AM,12:00:00,01/01/38474 12:00:00 AM,9:00AM,22-2-20,Burning solid waste in open fires,Responsible By Determination,$1500.00,$20.00,$150.00,$10.00,$0.00,$1680.00,PAID IN FULL,0.0,0,"2566 GRAND BLVD\nDetroit, MI\n(42.363182370000...","743 Beaubien\nDetroit, MI 48226\n(42.333730630..."
1,19800,05000025DAH,Department of Public Works,"JACKSON, RAECHELLE",19014,ASHTON,20501,HEYDEN,DETROIT,MI,48219,NaN,NaN,01/01/38383 12:00:00 AM,10:15:00,01/01/38425 12:00:00 AM,1:30PM,22-2-22,Bulk solid waste deposited more than 24 hours ...,Not responsible By Determination,$100.00,$20.00,$10.00,$10.00,$0.00,$140.00,NO PAYMENT APPLIED,0.0,0,"19014 ASHTON\nDetroit, MI\n(42.429390762000025...","20501 HEYDEN\nDETROIT, MI 48219\n(42.442177633..."
2,19804,05000026DAH,Department of Public Works,"TALTON, CAROL ANN",18735,STAHELIN,18735,STAHELI N,DETROIT,MI,48219,NaN,NaN,01/01/38383 12:00:00 AM,10:35:00,01/01/38425 12:00:00 AM,1:30PM,22-2-22,Bulk solid waste deposited more than 24 hours ...,Responsible By Determination,$100.00,$20.00,$10.00,$10.00,$0.00,$140.00,PAID IN FULL,0.0,0,"18735 STAHELIN\nDetroit, MI\n(42.4287074590000...","18735 STAHELI N\nDETROIT, MI 48219\n(42.428707..."
3,20208,05000027DAH,Department of Public Works,"BONNER, DARRYL E.",20125,MONICA,25335,PEEKSKILL,SOUTHFIELD,MI,48043,NaN,NaN,01/01/38385 12:00:00 AM,10:45:00,01/01/38422 12:00:00 AM,1:30PM,22-2-45,Violation of time limit for approved container...,Responsible By Default,$100.00,$20.00,$10.00,$10.00,$0.00,$140.00,NO PAYMENT APPLIED,0.0,0,"20125 MONICA\nDetroit, MI\n(42.44169828400004,...","25335 PEEKSKILL\nSOUTHFIELD, MI 48043\n(42.475..."
4,20211,05000028DAH,Department of Public Works,"GREGORY, JAMES LEE",17397,PRAIRIE,17397,PRAIRIE,DETROIT,MI,48221,NaN,NaN,01/01/38385 12:00:00 AM,11:10:00,01/01/38422 12:00:00 AM,1:30PM,22-2-22,Bulk solid waste deposited more than 24 hours ...,Responsible By Default,$100.00,$20.00,$10.00,$10.00,$0.00,$140.00,PAID IN FULL,0.0,0,"17397 PRAIRIE\nDetroit, MI\n(42.42031769500005...","17397 PRAIRIE\nDETROIT, MI 48221\n(42.42031769..."


A quick inspection shows that the following columns could be useful in blight prediction. The rationale is that these features reflect the nature and the seriousness of a violation. The occurences and the severity of the violations associated with a building should be strong indicators of blight.

In [92]:
cleandedViolations = pd.concat([violations['AgencyName'], violations['ViolationCode'], violations['FineAmt'], violations['StateFee'], violations['CleanUpCost'], violations['JudgmentAmt'] ,violations['PaymentStatus'], violations['ViolationCategory'], violations['ViolationAddress']], axis=1)
cleandedViolations.head()

,AgencyName,ViolationCode,FineAmt,StateFee,CleanUpCost,JudgmentAmt,PaymentStatus,ViolationCategory,ViolationAddress
0,Department of Public Works,22-2-20,$1500.00,$10.00,$0.00,$1680.00,PAID IN FULL,0,"2566 GRAND BLVD\nDetroit, MI\n(42.363182370000..."
1,Department of Public Works,22-2-22,$100.00,$10.00,$0.00,$140.00,NO PAYMENT APPLIED,0,"19014 ASHTON\nDetroit, MI\n(42.429390762000025..."
2,Department of Public Works,22-2-22,$100.00,$10.00,$0.00,$140.00,PAID IN FULL,0,"18735 STAHELIN\nDetroit, MI\n(42.4287074590000..."
3,Department of Public Works,22-2-45,$100.00,$10.00,$0.00,$140.00,NO PAYMENT APPLIED,0,"20125 MONICA\nDetroit, MI\n(42.44169828400004,..."
4,Department of Public Works,22-2-22,$100.00,$10.00,$0.00,$140.00,PAID IN FULL,0,"17397 PRAIRIE\nDetroit, MI\n(42.42031769500005..."


There are some columns that involves money amounts. Let's convert them into numerical values for later ease.

In [93]:
def toDollar(dollar):
    try:
        return float(dollar[1:])
    except:
        return 0

for col in ['StateFee', 'CleanUpCost', 'JudgmentAmt', 'FineAmt']:
    cleandedViolations[col] = cleandedViolations.apply(lambda row: toDollar(row[col]), axis=1)

cleandedViolations.head()

,AgencyName,ViolationCode,FineAmt,StateFee,CleanUpCost,JudgmentAmt,PaymentStatus,ViolationCategory,ViolationAddress
0,Department of Public Works,22-2-20,1500.0,10.0,0.0,1680.0,PAID IN FULL,0,"2566 GRAND BLVD\nDetroit, MI\n(42.363182370000..."
1,Department of Public Works,22-2-22,100.0,10.0,0.0,140.0,NO PAYMENT APPLIED,0,"19014 ASHTON\nDetroit, MI\n(42.429390762000025..."
2,Department of Public Works,22-2-22,100.0,10.0,0.0,140.0,PAID IN FULL,0,"18735 STAHELIN\nDetroit, MI\n(42.4287074590000..."
3,Department of Public Works,22-2-45,100.0,10.0,0.0,140.0,NO PAYMENT APPLIED,0,"20125 MONICA\nDetroit, MI\n(42.44169828400004,..."
4,Department of Public Works,22-2-22,100.0,10.0,0.0,140.0,PAID IN FULL,0,"17397 PRAIRIE\nDetroit, MI\n(42.42031769500005..."


Once again we process the coordinates and obtain the bloc IDs.

In [95]:
violationsCoords = cleandedViolations.apply(lambda row:parseAddress(row['ViolationAddress']),axis=1)
violationsCoords.columns = ['lat', 'lng']

violationsCoords['lat'] = pd.to_numeric(violationsCoords['lat'], errors='raise') 
violationsCoords['lng'] = pd.to_numeric(violationsCoords['lng'], errors='raise') 

violationsCoords['lat'] = violationsCoords.apply(lambda row: np.round(row['lat'], 5), axis=1)
violationsCoords['lng'] = violationsCoords.apply(lambda row: np.round(row['lng'], 5), axis=1)

violations['bloc_id'] = violations.apply(lambda row: getBlocs(row['lat'], row['lng']), axis=1)

violations = pd.concat([cleandedViolations, violationsCoords], axis=1)

Another interesting feature is the violation code. It is of format `22-2-20`. We break it up into three parts to better capture the types of the violations.

In [12]:
def getViolationsCodeDetails(violationCode):
    details = violationCode.split('-')
    data = {}
    for l in range(3):
        if len(details) > l and details[l] is not None:
            data['violationCode-{0}'.format(l)] = details[l]
    
    if len(data) > 0:
        return pd.Series(data)
    else:
        pass
    
violations = violations.merge(violations.ViolationCode.apply(lambda row: getViolationsCodeDetails(row)), left_index=True, right_index=True)
violations.head()

,AgencyName,ViolationCode,FineAmt,StateFee,CleanUpCost,JudgmentAmt,PaymentStatus,ViolationCategory,ViolationAddress,lat,lng,violationCode-0,violationCode-1,violationCode-2
0,Department of Public Works,22-2-20,1500.0,10.0,0.0,1680.0,PAID IN FULL,0,"2566 GRAND BLVD\nDetroit, MI\n(42.363182370000...",42.363182,-83.091677,22,2,20
1,Department of Public Works,22-2-22,100.0,10.0,0.0,140.0,NO PAYMENT APPLIED,0,"19014 ASHTON\nDetroit, MI\n(42.429390762000025...",42.429391,-83.220394,22,2,22
2,Department of Public Works,22-2-22,100.0,10.0,0.0,140.0,PAID IN FULL,0,"18735 STAHELIN\nDetroit, MI\n(42.4287074590000...",42.428707,-83.227548,22,2,22
3,Department of Public Works,22-2-45,100.0,10.0,0.0,140.0,NO PAYMENT APPLIED,0,"20125 MONICA\nDetroit, MI\n(42.44169828400004,...",42.441698,-83.145018,22,2,45
4,Department of Public Works,22-2-22,100.0,10.0,0.0,140.0,PAID IN FULL,0,"17397 PRAIRIE\nDetroit, MI\n(42.42031769500005...",42.420318,-83.145328,22,2,22


Next, we will group the incidents by the bloc IDs and compute each group's statistics using `agg()` function. For money amount, we are looking for the sum. For other categorical features, one simple approach is to just take one of the values as the aggregated value. And we will want to count the total number of violations related to a building.

In [456]:
violations = violations.drop(columns=['ViolationCode', 'violationCode-1', 'violationCode-2', 'ViolationAddress'], axis=1)
violations['nb_violations'] = 0

groupedViolations = violations.groupby('bloc_id', as_index=False)
groupedViolationsAgg = groupedViolations.agg({
        'AgencyName': 'first',
        'violationCode-0': 'first',
        'FineAmt': 'sum',
        'StateFee': 'sum',
        'CleanUpCost': 'sum',
        'JudgmentAmt': 'sum',
        'PaymentStatus': 'first',
        'ViolationCategory': 'first',
        'nb_violations': 'count'
    })

groupedViolationsAgg.head()

,bloc_id,CleanUpCost,FineAmt,nb_violations,AgencyName,JudgmentAmt,StateFee,violationCode-0,ViolationCategory,PaymentStatus
0,0.0,0.0,550.0,2,Department of Public Works,665.0,20.0,22,0,NO PAYMENT APPLIED
1,1.0,0.0,50.0,1,Department of Public Works,85.0,10.0,9,0,NO PAYMENT APPLIED
2,28.0,0.0,200.0,1,Department of Public Works,250.0,10.0,22,0,NO PAYMENT APPLIED
3,32.0,0.0,1800.0,5,Building and Safety Engineering Department,2130.0,50.0,9,0,PAID IN FULL
4,37.0,320.0,3500.0,7,Building and Safety Engineering Department,4060.0,70.0,9,0,NO PAYMENT APPLIED


Very similarly, we will process the crimes dataset and the 311 calls dataset. The procedure for parsing the coordinates and obtaining bloc IDs are the same. Due to the limit on lines of code, those codes are omitted. Instead the processed datasets are loaded directly.

In [467]:
crimes = pd.read_csv('data/crimes_blocs.csv')
crimes['nb_crimes'] = 0

groupedCrimes = crimes.groupby('bloc_id', as_index=False)
groupedCrimesAgg = groupedCrimes.agg({
        'STATEOFFENSEFILECLASS': 'last',
        'nb_crimes': 'count'
    })

In [459]:
calls = pd.read_csv('data/311_blocs.csv')
calls['nb_calls'] = 0

groupedCalls = calls.groupby('bloc_id', as_index=False)
groupedCallsAgg = groupedCalls.agg({
        'issue_type': 'last',
        'rating': 'min',
        'ticket_status': 'last',
        'nb_calls': 'count'
    })

## Constructing Final Dataset

As the final step, we will combine everything we've obtained so far into a dataset that can be used by machine learning models.

First, we will merge the labels with corresponding features.

In [530]:
data = pd.merge(trainDF, groupedViolationsAgg, on='bloc_id', how='left')
data = pd.merge(data, groupedCallsAgg, on='bloc_id', how='left')
data = pd.merge(data, groupedCrimesAgg, on='bloc_id', how='left')

Next, we need to fill columns that contain NAs with reasonable values. For numerical features like counts or money amount, it's obvious that we should replaces NAs with 0s. For other categorical features, we can use -1s to indicate "missing".

In [ ]:
for col in ['CleanUpCost', 'FineAmt', 'nb_violations', 'nb_calls', 'nb_crimes', 'JudgmentAmt', 'StateFee']:
    data[col] = data[col].fillna(0)
for col in ['AgencyName', 'violationCode-0', 'ViolationCategory', 'PaymentStatus', 'STATEOFFENSEFILECLASS', 'rating', 'ticket_status', 'issue_type']:
    data[col] = data[col].fillna(-1)

Here's what our final dataset look like:

In [540]:
data.head()

,bloc_id,label,CleanUpCost,FineAmt,nb_violations,AgencyName,JudgmentAmt,StateFee,violationCode-0,ViolationCategory,PaymentStatus,rating,nb_calls,ticket_status,issue_type,STATEOFFENSEFILECLASS,nb_crimes
0,80884.0,blighted,0.0,600.0,2.0,Detroit Police Department,720.0,20.0,9,0.0,NO PAYMENT APPLIED,-1.0,0.0,-1,-1,-1.0,0.0
1,52539.0,blighted,0.0,2550.0,9.0,Building and Safety Engineering Department,3075.0,90.0,9,0.0,NO PAYMENT APPLIED,-1.0,0.0,-1,-1,35001.0,1.0
2,199492.0,blighted,0.0,0.0,0.0,-1,0.0,0.0,-1,-1.0,-1,-1.0,0.0,-1,-1,-1.0,0.0
3,199493.0,blighted,0.0,0.0,0.0,-1,0.0,0.0,-1,-1.0,-1,-1.0,0.0,-1,-1,-1.0,0.0
4,199494.0,blighted,0.0,0.0,0.0,-1,0.0,0.0,-1,-1.0,-1,-1.0,0.0,-1,-1,-1.0,0.0


## Model Training

In this section, we will train our model on the generated dataset and evaluate its performance. As this part is not the focus of this tutorial, we will only train a random forest model for simplicity.

In [5]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

Since we have several categorical columns in our dataset, it helps to use a label encoder to convert the values into integers for later ease with the model training.

In [6]:
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns

    def fit(self,X,y=None):
        return self

    def transform(self,X):
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)
    
mcle = MultiColumnLabelEncoder(columns = ['label','AgencyName', 'violationCode-0', 'PaymentStatus', 'STATEOFFENSEFILECLASS', 'ticket_status', 'issue_type']) 
data = mcle.fit_transform(data)

Next, we will split the dataset into training and testing sets.

In [9]:
from sklearn.cross_validation import train_test_split

data_labels = data['label']
data_features = data.drop('label', axis=1)
data_features = data_features.drop('bloc_id', axis=1)

feat_train, feat_test, out_train, out_test = train_test_split(data_features, data_labels, test_size=0.2, random_state=1027)

Now let's train a random forest classifier on the training set and evaluate its performance on the testing set.

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import cross_validation

model = RandomForestClassifier(n_estimators=10, max_features=5, max_depth=6)
model = model.fit(feat_train, out_train)

scores = cross_validation.cross_val_score(model, feat_train, out_train, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.72 (+/- 0.01)


In [12]:
forest_test_results = model.predict(feat_test)
accuracy_score(out_test, np.around(forest_test_results))

0.73147410358565734

One advantage of random forests is that we can immediately inspect the feature importances using the trained model.

Based on feature importances, we can see that the number of violations and the amount of fines/costs are very strong predictors for the blights. Agency name is also a strong predictor because it reveals the nature of the incident, as different issues will normally be handled by different agencies/authorities.

In [13]:
importances = model.feature_importances_
std = np.std([tree.feature_importances_ for tree in model.estimators_], axis=0)
indices = np.argsort(importances)[::-1]

print("Top features:")

col_labels = feat_test.columns.values.tolist()
for var in range(0, len(col_labels)):
    print("%d. %s (%f)" % (var, col_labels[var], importances[indices[var]]))

Top features:
0. CleanUpCost (0.268485)
1. FineAmt (0.219016)
2. nb_violations (0.091803)
3. AgencyName (0.090707)
4. JudgmentAmt (0.075536)
5. StateFee (0.075392)
6. violationCode-0 (0.048935)
7. ViolationCategory (0.044099)
8. PaymentStatus (0.022746)
9. rating (0.018151)
10. nb_calls (0.017838)
11. ticket_status (0.010091)
12. issue_type (0.010000)
13. STATEOFFENSEFILECLASS (0.005431)
14. nb_crimes (0.001667)
15. total_charges (0.000103)


## Future Works

- For now we are clustering locations purely based on coordinates, while the datasets also contains addresses of these locations. We can preprocess the address field and use some sort of fuzzy text matching as another criteria for determining whether two coordinates should be mapped to the same building.
- An incidient affeacts not only the building but also the whole neighborhood. The effect is something that decays with the distance. For now we are only taking into account the impact of an incident to the building that it is associated with. We can try expanding that impact to all the nearby buildings within a certain distance. Furthermore we can define different values to use for the impact radius to more accurately capture the impact pattern.
- To make our model more useful to urban planners, we can divide the whole Detroit area into grids and predict a "blight factor" for all the cells. Then we can build a heat map to interactively display the data.

## References

- [More on DBSCAN](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html)
- [More on how to handle imbalanced datasets](https://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset/)
- [Example of blight maps](https://www.citylab.com/equity/2014/05/we-now-have-highly-detailed-maps-of-detroits-blight/371762/)